In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

In [2]:
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import matthews_corrcoef
from sklearn.model_selection import train_test_split

In [3]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, cross_val_predict

C:\Users\User\anaconda3\Lib\site-packages\dask\dataframe\_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


In [4]:
df = pd.read_csv('protine_paac.csv')
df.head()

,0,1.694633037,0.1,1.694633037.1,0.847316518,0.2,0.3,1.694633037.2,1.694633037.3,0.4,...,0.6,0.847316518.2,0.847316518.3,1.694633037.5,1.694633037.6,0.847316518.4,0.847316518.5,0.847316518.6,0.07721199,0.075471492
0,0,0.000000,0.000000,0.838278,0.000000,0.000000,0.0,2.514834,0.838278,0.000000,...,1.676556,0.838278,1.676556,0.000000,1.676556,0.000000,1.676556,0.000000,0.063878,0.097844
1,0,0.824192,0.824192,0.000000,0.824192,0.000000,0.0,1.648385,1.648385,0.000000,...,0.824192,0.824192,0.000000,0.824192,0.824192,0.000000,0.824192,0.000000,0.074179,0.101629
2,0,2.485999,0.000000,0.000000,0.000000,0.828666,0.0,0.828666,0.828666,0.828666,...,0.828666,0.000000,0.828666,0.828666,0.828666,0.828666,0.000000,1.657333,0.088327,0.083007
3,0,2.461841,0.820614,0.000000,0.820614,0.000000,0.0,2.461841,0.000000,0.820614,...,0.820614,0.000000,1.641227,0.000000,0.820614,0.000000,0.000000,2.461841,0.113222,0.066164
4,0,0.000000,0.000000,0.000000,0.848823,0.848823,0.0,1.697645,2.546468,0.000000,...,0.000000,0.848823,1.697645,1.697645,0.848823,0.000000,0.000000,3.395290,0.067183,0.083995


In [5]:
y = df['0']
x = df.drop('0', axis=1) 

In [6]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.3)

In [7]:
total_Metics = []
total_Metics = pd.DataFrame(total_Metics)
total_Metics['Classifier'] = 'Classifier'
total_Metics['Accuracy'] = 'Accuracy'
total_Metics['mcc'] = 'mcc'
# total_Metics['auc'] = 'auc'
total_Metics['Kappa'] = 'Kappa'
total_Metics['precision'] = 'precision'
total_Metics['recall'] = 'recall'
total_Metics['f1'] = 'f1'
total_Metics['sensitivity'] = 'sensitivity'
total_Metics['specificity'] = 'specificity'

models = [RandomForestClassifier(n_estimators = 200, max_depth = 10),
          XGBClassifier(n_estimators = 200,max_depth = 10, learning_rate = 0.1),
          LGBMClassifier(learning_rate = 0.1,max_depth = 10,random_state = 50),
          GradientBoostingClassifier(n_estimators = 200, learning_rate = 0.5, random_state = 50),
          AdaBoostClassifier(n_estimators = 200, learning_rate = 0.1, random_state = 50)]
cv = KFold(n_splits=5, random_state=1, shuffle=True)
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  # evaluate model
  # scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  # model.fit(xtrain, ytrain)
  # pred = model.predict(xtest)
  pred = cross_val_predict(model, xtrain, ytrain, cv=cv, n_jobs=-1)

  # cm1 = confusion_matrix(y, y_pred)
  # report performance
  Accuracy = accuracy_score(ytrain, pred)
  mcc = matthews_corrcoef(ytrain, pred)
  cm1 = confusion_matrix(ytrain, pred)
  kappa = cohen_kappa_score(ytrain, pred)
  f1 = f1_score(ytrain, pred)
  precision_score = precision_score(ytrain, pred)
  recall_score = recall_score(ytrain, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  # y_pred = np.argmax(y_pred, axis=0)
  # auc = roc_auc_score(y, y_pred, multi_class='ovr')
  total_Metics.loc[len(total_Metics.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

print(total_Metics)

                                          Classifier  Accuracy       mcc  \
0  RandomForestClassifier(max_depth=10, n_estimat...  0.636911  0.091857   
1  XGBClassifier(base_score=None, booster=None, c...  0.617854  0.107330   
2      LGBMClassifier(max_depth=10, random_state=50)  0.599799  0.069482   
3  GradientBoostingClassifier(learning_rate=0.5, ...  0.619860  0.134094   
4  AdaBoostClassifier(learning_rate=0.1, n_estima...  0.634905  0.113003   

      Kappa  precision    recall        f1  sensitivity  specificity  
0  0.066383   0.500000  0.129834  0.206140     0.925984     0.129834  
1  0.101749   0.458150  0.287293  0.353141     0.806299     0.287293  
2  0.066543   0.422594  0.279006  0.336106     0.782677     0.279006  
3  0.131144   0.468635  0.350829  0.401264     0.773228     0.350829  
4  0.095710   0.493243  0.201657  0.286275     0.881890     0.201657  


In [8]:
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)
x_ros, y_ros = ros.fit_resample(xtrain, ytrain)
total_Metics = []
total_Metics = pd.DataFrame(total_Metics)
total_Metics['Classifier'] = 'Classifier'
total_Metics['Accuracy'] = 'Accuracy'
total_Metics['mcc'] = 'mcc'
# total_Metics['auc'] = 'auc'
total_Metics['Kappa'] = 'Kappa'
total_Metics['precision'] = 'precision'
total_Metics['recall'] = 'recall'
total_Metics['f1'] = 'f1'
total_Metics['sensitivity'] = 'sensitivity'
total_Metics['specificity'] = 'specificity'

models = [RandomForestClassifier(n_estimators = 200, max_depth = 10),
          XGBClassifier(n_estimators = 200,max_depth = 10, learning_rate = 0.1),
          LGBMClassifier(learning_rate = 0.1,max_depth = 10,random_state = 50),
          GradientBoostingClassifier(n_estimators = 200, learning_rate = 0.5, random_state = 50),
          AdaBoostClassifier(n_estimators = 200, learning_rate = 0.1, random_state = 50)]
cv = KFold(n_splits=5, random_state=1, shuffle=True)
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  # evaluate model
  # scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  # model.fit(xtrain, ytrain)
  # pred = model.predict(xtest)
  pred = cross_val_predict(model, x_ros, y_ros, cv=cv, n_jobs=-1)

  # cm1 = confusion_matrix(y, y_pred)
  # report performance
  Accuracy = accuracy_score(y_ros, pred)
  mcc = matthews_corrcoef(y_ros, pred)
  cm1 = confusion_matrix(y_ros, pred)
  kappa = cohen_kappa_score(y_ros, pred)
  f1 = f1_score(y_ros, pred)
  precision_score = precision_score(y_ros, pred)
  recall_score = recall_score(y_ros, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  # y_pred = np.argmax(y_pred, axis=0)
  # auc = roc_auc_score(y, y_pred, multi_class='ovr')
  total_Metics.loc[len(total_Metics.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

print(total_Metics)

                                          Classifier  Accuracy       mcc  \
0  RandomForestClassifier(max_depth=10, n_estimat...  0.743307  0.486856   
1  XGBClassifier(base_score=None, booster=None, c...  0.755906  0.512730   
2      LGBMClassifier(max_depth=10, random_state=50)  0.729921  0.461333   
3  GradientBoostingClassifier(learning_rate=0.5, ...  0.749606  0.502453   
4  AdaBoostClassifier(learning_rate=0.1, n_estima...  0.597638  0.195330   

      Kappa  precision    recall        f1  sensitivity  specificity  
0  0.486614   0.735878  0.759055  0.747287     0.727559     0.759055  
1  0.511811   0.741456  0.785827  0.762997     0.725984     0.785827  
2  0.459843   0.712828  0.770079  0.740348     0.689764     0.770079  
3  0.499213   0.724187  0.806299  0.763040     0.692913     0.806299  
4  0.195276   0.595385  0.609449  0.602335     0.585827     0.609449  
